In [1]:
%%capture
!pip install neurokit2 timm

In [ ]:
!gdown 1f0vsok44_ZwKnLK2AGRqYyw0FmYcxrCT # test.zip
!gdown 1zXWuODLYE3UFgdUqn4HCnF93Ubg48sAm # sample_submissions к открытому тесту

!gdown 1-Rnz3VbbqD1GWqqEDRcrQuzO_itkmjsG # senet val_0.990

!gdown 1-_-wij1PVxCuyS-Hmn6hl37dvlKBoc0L # senet focal loss3 val 0.974

!gdown 1-pDRvXXpK7whRnl7uPWYZ1zidZG6GRlf # senet focal loss val 0.970
!gdown 1-Wv-34aEc6R7-EPcrVhC02qmzqQkW-PP # new rp senet val_0.981 focal loss

!gdown 1-HQ-Xa3CHptHOE1N0YaOE-vPfnryHq2H # ecgNet_simple_r_f1 0.98

!unzip test.zip

In [4]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os

import torch
import timm
import torch.nn as nn
from scipy.signal import resample
import neurokit2 as nk

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#### Подгрузка SEResNeXt из библиотеки timm и добавление туда разных предобученных весов

Все модели обученны через [focal loss](https://www.hindawi.com/journals/jhe/2022/9370517/#:~:text=Ensemble%20Classifier%20with-,Focal%20Loss,-for%20Automatic%20Arrhythmia) с разным gamma

In [4]:
%%capture
model1 = timm.create_model('gluon_seresnext101_32x4d' , pretrained=True)

model1.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model1.fc = torch.nn.Linear(in_features=2048, out_features=7, bias=True)

model1.load_state_dict(torch.load('/content/se_resnext_simpRP_focal_loss3_f1:0.974_5000.pth', map_location=torch.device(device)))

for param in model1.parameters():
     param.requires_grad = False

model1.to(device)
model1.eval()
None

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name gluon_seresnext101_32x4d to current seresnext101_32x4d.gluon_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/196M [00:00<?, ?B/s]

In [5]:
model2 = timm.create_model('gluon_seresnext101_32x4d' , pretrained=True)

model2.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model2.fc = torch.nn.Linear(in_features=2048, out_features=7, bias=True)

model2.load_state_dict(torch.load('/content/se_resnext_simpRP_focal_loss_f1:0.970_5000.pth', map_location=torch.device(device)))

for param in model2.parameters():
     param.requires_grad = False

model2.to(device)
model2.eval()
None

In [6]:
model3 = timm.create_model('gluon_seresnext101_32x4d' , pretrained=True)

model3.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model3.fc = torch.nn.Linear(in_features=2048, out_features=7, bias=True)

model3.load_state_dict(torch.load('/content/se_resnext_simpRP_focal_loss_f1:0.981_5000.pth', map_location=torch.device(device)))

for param in model3.parameters():
     param.requires_grad = False

model3.to(device)
model3.eval()
None

In [7]:
model4 = timm.create_model('gluon_seresnext101_32x4d' , pretrained=True)

model4.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model4.fc = torch.nn.Linear(in_features=2048, out_features=7, bias=True)

model4.load_state_dict(torch.load('/content/se_resnext_simple_Rp_f1:0.990_5000_best.pth', map_location=torch.device(device)))

for param in model4.parameters():
     param.requires_grad = False

model4.to(device)
model4.eval()
None

#### Уникальная ECGNet созданная на основе [статьи](https://arxiv.org/pdf/2012.05510.pdf)

In [8]:
from collections import OrderedDict

class ECGNet(nn.Module):
  def __init__(self):
    super(ECGNet, self).__init__()
    #layer1
    self.layer1_conv2d = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1, 25), stride=(1, 2), bias=True)


    #layer2
    self.layer2_conv2d = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm2d(num_features=32)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv2d(32, 64, kernel_size=(1, 15), stride=(1, 1), bias=True)),
        ("bn2", nn.BatchNorm2d(num_features=64)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv2d(64, 64, kernel_size=(1, 15), stride=(1, 2),  bias=True)),
        ("bn3", nn.BatchNorm2d(num_features=64)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv2d(64, 32, kernel_size=(1, 15), stride=(1, 1), bias=True)),
    ]))
    self.layer2_seModule = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv2d(32, 16, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv2d(16, 32, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    #layer3
    self.layer3_conv2d_block1 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm2d(num_features=32)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv2d(32, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=True)),
        ("bn2", nn.BatchNorm2d(num_features=64)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=True)),
        ("bn3", nn.BatchNorm2d(num_features=64)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv2d(64, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=True)),
    ]))
    self.layer3_seModule_block1 = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv2d(32, 16, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv2d(16, 32, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    self.layer3_conv2d_block2 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm2d(num_features=32)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv2d(32, 64, kernel_size=(5, 1), padding=(2, 0), bias=True)),
        ("bn2", nn.BatchNorm2d(num_features=64)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv2d(64, 64, kernel_size=(5, 1), padding=(2, 0), bias=True)),
        ("bn3", nn.BatchNorm2d(num_features=64)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv2d(64, 32, kernel_size=(5, 1), padding=(2, 0), bias=True)),
    ]))
    self.layer3_seModule_block2 = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv2d(32, 16, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv2d(16, 32, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    self.layer3_conv2d_block3 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm2d(num_features=32)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv2d(32, 64, kernel_size=(7, 1), padding=(3, 0), bias=True)),
        ("bn2", nn.BatchNorm2d(num_features=64)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv2d(64, 64, kernel_size=(7, 1), padding=(3, 0), bias=True)),
        ("bn3", nn.BatchNorm2d(num_features=64)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv2d(64, 32, kernel_size=(7, 1), padding=(3, 0), bias=True)),
    ]))
    self.layer3_seModule_block3 = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv2d(32, 16, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv2d(16, 32, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    #layer4
    self.layer4_conv1d_short_block1 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm1d(num_features=384)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv1d(384, 384, kernel_size=3, stride=9, bias=True)),
    ]))

    self.layer4_conv1d_block1 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm1d(num_features=384)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv1d(384, 768, kernel_size=3, stride=2, bias=True)),
        ("bn2", nn.BatchNorm1d(num_features=768)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv1d(768, 768, kernel_size=3, stride=1, bias=True)),
        ("bn3", nn.BatchNorm1d(num_features=768)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv1d(768, 1536, kernel_size=3, stride=2, bias=True)),
        ("bn4", nn.BatchNorm1d(num_features=1536)),
        ("act4", nn.ReLU()),
        ("cn4", nn.Conv1d(1536, 384, kernel_size=3, stride=2, bias=True)),
    ]))
    self.layer4_seModule_block1 = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv1d(384, 48, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv1d(48, 384, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    self.layer4_conv1d_short_block2 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm1d(num_features=384)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv1d(384, 384, kernel_size=5, stride=9, bias=True)),
    ]))

    self.layer4_conv1d_block2 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm1d(num_features=384)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv1d(384, 768, kernel_size=5, stride=2, padding=2, bias=True)),
        ("bn2", nn.BatchNorm1d(num_features=768)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv1d(768, 768, kernel_size=5, stride=2, padding=1, bias=True)),
        ("bn3", nn.BatchNorm1d(num_features=768)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv1d(768, 1536, kernel_size=5, stride=1, padding=2, bias=True)),
        ("bn4", nn.BatchNorm1d(num_features=1536)),
        ("act4", nn.ReLU()),
        ("cn4", nn.Conv1d(1536, 384, kernel_size=5, stride=2, padding=1, bias=True)),
    ]))
    self.layer4_seModule_block2 = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv1d(384, 48, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv1d(48, 384, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    self.layer4_conv1d_short_block3 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm1d(num_features=384)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv1d(384, 384, kernel_size=7, stride=9, bias=True)),
    ]))

    self.layer4_conv1d_block3 = nn.Sequential(OrderedDict([
        ("bn1", nn.BatchNorm1d(num_features=384)),
        ("act1", nn.ReLU()),
        ("cn1", nn.Conv1d(384, 768, kernel_size=7, stride=2, padding=2, bias=True)),
        ("bn2", nn.BatchNorm1d(num_features=768)),
        ("act2", nn.ReLU()),
        ("cn2", nn.Conv1d(768, 768, kernel_size=7, stride=2, padding=1, bias=True)),
        ("bn3", nn.BatchNorm1d(num_features=768)),
        ("act3", nn.ReLU()),
        ("cn3", nn.Conv1d(768, 1536, kernel_size=7, stride=1, padding=3, bias=True)),
        ("bn4", nn.BatchNorm1d(num_features=1536)),
        ("act4", nn.ReLU()),
        ("cn4", nn.Conv1d(1536, 384, kernel_size=7, stride=2, padding=2, bias=True)),
    ]))
    self.layer4_seModule_block3 = nn.Sequential(OrderedDict([
        ("fc1", nn.Conv1d(384, 48, kernel_size=1, bias=True)),
        ("act", nn.ReLU()),
        ("fc2", nn.Conv1d(48, 384, kernel_size=1, bias=True)),
        ("gate", nn.Sigmoid())
    ]))

    self.layer5_avg_pool1 = nn.AvgPool1d(kernel_size=10)
    self.layer5_avg_pool2 = nn.AvgPool1d(kernel_size=10)
    self.layer5_avg_pool3 = nn.AvgPool1d(kernel_size=10)

    self.fc = nn.Sequential(OrderedDict([
        ("ln1", nn.Linear(1152, 288)),
        ("dp", nn.Dropout(p=0.2)),
        ("act", nn.ReLU()),
        ("ln2", nn.Linear(288, 7))
    ]))

  def forward(self, x):
    #layer1
    x = self.layer1_conv2d(x)

    #layer2
    x = self.layer2_conv2d(x)
    u = x
    x = x.view(x.size(0), x.size(1), -1).mean(-1).view(x.size(0), x.size(1), 1, 1)
    x = self.layer2_seModule(x)
    x = u * x

    #layer3
    x1 = self.layer3_conv2d_block1(x)
    u1 = x1
    x1 = x1.view(x1.size(0), x1.size(1), -1).mean(-1).view(x1.size(0), x1.size(1), 1, 1)
    x1 = self.layer3_seModule_block1(x1)
    x1 = u1 * x1

    x2 = self.layer3_conv2d_block2(x)
    u2 = x2
    x2 = x2.view(x2.size(0), x2.size(1), -1).mean(-1).view(x2.size(0), x2.size(1), 1, 1)
    x2 = self.layer3_seModule_block2(x2)
    x2 = u2 * x2

    x3 = self.layer3_conv2d_block3(x)
    u3 = x3
    x3 = x3.view(x3.size(0), x3.size(1), -1).mean(-1).view(x3.size(0), x3.size(1), 1, 1)
    x3 = self.layer3_seModule_block3(x3)
    x3 = u3 * x3

    #layer4
    x1 = torch.flatten(x1, start_dim=1, end_dim=2)
    x2 = torch.flatten(x2, start_dim=1, end_dim=2)
    x3 = torch.flatten(x3, start_dim=1, end_dim=2)

    x1_short = self.layer4_conv1d_short_block1(x1)

    x1 = self.layer4_conv1d_block1(x1)
    u1 = x1
    x1 = x1.view(x1.size(0), x1.size(1), -1).mean(-1).view(x1.size(0), x1.size(1), 1, 1).flatten(2, 3)
    x1 = self.layer4_seModule_block1(x1)
    x1 = u1 * x1
    x1 = x1 + x1_short

    x2_short = self.layer4_conv1d_short_block2(x2)

    x2 = self.layer4_conv1d_block2(x2)
    u2 = x2
    x2 = x2.view(x2.size(0), x2.size(1), -1).mean(-1).view(x2.size(0), x2.size(1), 1, 1).flatten(2, 3)
    x2 = self.layer4_seModule_block2(x2)
    x2 = u2 * x2
    x2 = x2 + x2_short

    x3_short = self.layer4_conv1d_short_block3(x3)

    x3 = self.layer4_conv1d_block3(x3)
    u3 = x3
    x3 = x3.view(x3.size(0), x3.size(1), -1).mean(-1).view(x3.size(0), x3.size(1), 1, 1).flatten(2, 3)
    x3 = self.layer4_seModule_block3(x3)
    x3 = u3 * x3
    x3 = x3 + x3_short

    x1 = self.layer5_avg_pool1(x1)
    x2 = self.layer5_avg_pool2(x2)
    x3 = self.layer5_avg_pool3(x3)

    x = torch.cat((x1, x2, x3), dim=1).flatten(1)

    x = self.fc(x)

    return x

In [9]:
model5 = ECGNet()

model5.load_state_dict(torch.load('/content/ecgNet_simple_r_f1:0.98_5000.pth', map_location=torch.device(device)))

for param in model3.parameters():
     param.requires_grad = False

model5.to(device)
model5.eval()
None

In [10]:
def get_predicts(data_path, sample_submission=None):
    """
    data_path: путь к директроии с сигналами
    sample_submission: путь датафрейму с колонками record_name myocard_type который будет заполняться, если None создается на основе data_path
    """

    predict = pd.DataFrame(columns=['record_name', 'myocard_type'])

    # создание датафрейма для предсказаний
    if sample_submission is None:
        predict.record_name = [record[:-4] for record in os.listdir(data_path) if record[-4:] == '.npy']
    else:
        test = pd.read_csv(sample_submission)
        predict.record_name = test.record_name

    for name in tqdm(predict.record_name):
        data = np.load(f'{data_path}/{name}.npy')

        # отчистка всез каналов ЭКГ
        data = np.apply_along_axis(lambda x: nk.ecg_clean(x, sampling_rate=500), axis=1, arr=data)

        # Нахождение среднего числа R-пиков
        peaks = [nk.ecg_findpeaks(data[i])['ECG_R_Peaks'] for i in range(12)]
        peaks_count = [(i, len(i)) for i in peaks]

        peaks = max(peaks_count, key=lambda x: x[1])[0]

        signals = []
        for count, i in enumerate(peaks):
            if count == 0:
                diff2 = abs(peaks[count + 1]- i)
                x = 0
                y = peaks[count + 1] - diff2 // 2
            elif count == len(peaks)-1:
                diff1 = abs(peaks[count - 1] - i)
                x = peaks[count - 1] + diff1 // 2
                y = 5000
            else:
                diff1 = abs(peaks[count - 1] - i)
                diff2 = abs(peaks[count + 1]- i)
                x = peaks[count - 1] + diff1 // 2
                y = peaks[count + 1] - diff2 // 2

            signal = torch.tensor(resample(data[:, x:y], 500, axis=1)).to(device)

            signals.append(signal)


        signals = torch.stack(signals)[:, None, :, :]

        # среднее взвешенное между всеми моделями
        logits = (model1(signals.float()) + model3(signals.float()) + model2(signals.float()) + model4(signals.float()) + model5(signals.float())) / 5

        peaks_logits = torch.argmax(logits, dim=1)

        ans = torch.mode(peaks_logits).values.item()

        predict.loc[predict.record_name == name, 'myocard_type'] = ans

    return predict

In [11]:
predict = get_predicts(data_path='test', sample_submission='test.csv')

  0%|          | 0/449 [00:00<?, ?it/s]

In [ ]:
predict.groupby('myocard_type').count()

In [7]:
def resample_predictions(predicts):
    id2label = {0: 'норма', 1: 'передне-перегородочный', 2: 'перегородочный', 3: 'нижний', 4: 'передний', 5: 'передне-боковой', 6: 'боковой'}

    myocards_name = ['перегородочный', 'передний', 'боковой', 'передне-боковой', 'передне-перегородочный', 'нижний', 'норма', 'record_name']

    ans = pd.DataFrame(columns=myocards_name)

    for name in predicts.record_name:
        myocard_label = id2label[predicts[predicts.record_name == name].myocard_type.values[0]]

        current_df = {k: [0] for k in myocards_name}
        current_df['record_name'], current_df[myocard_label] = [name], [1]

        ans = pd.concat([ans, pd.DataFrame(data=current_df)])

    return ans

In [8]:
predict = resample_predictions(predict)

In [ ]:
predict

In [10]:
predict.to_csv('predict.csv', index=False)